# Function Calling with Open Source LLMs (LLaMa-3, LangChain & Groq API)

### Table of Content

1. **Load LLMs**
2. **Define Tools**
3. **Link Tools with LLMs**
4. **Use LLM with Tools**
5. **Create Independent Decision Taking Agent**

### Installation

* **pip install langchain**


In [1]:
import dotenv
import os

dotenv.load_dotenv(dotenv.find_dotenv())

groq_api_key = os.environ["GROQ_API_KEY"]

## 1. Load LLMs

* Login to **https://console.groq.com** and create API Key.

### Groq Models

ID|	REQUESTS PER MINUTE|	REQUESTS PER DAY|	TOKENS PER MINUTE
-|-|-|-
llama3-70b-8192	|30	|14,400	|6,000
llama3-8b-8192	|30	|14,400	|30,000
gemma-7b-it	|30	|14,400	|15,000
mixtral-8x7b-32768	|30	|14,400	|5,000


In [2]:
from langchain_openai import ChatOpenAI

llama3 = ChatOpenAI(api_key=groq_api_key, 
                    base_url="https://api.groq.com/openai/v1",
                    model="llama3-8b-8192",
                   )

llama3

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f89d749dd80>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f89d749f490>, model_name='llama3-8b-8192', openai_api_key=SecretStr('**********'), openai_api_base='https://api.groq.com/openai/v1', openai_proxy='')

In [3]:
ai_msg = llama3.invoke("Hi! How are you?")

print(ai_msg.content)

I'm just an AI, I don't have emotions or personal experiences like humans do, so I don't have feelings or emotions. I exist solely to provide information, answer questions, and assist with tasks to the best of my ability. I'm here to help you with any questions or topics you'd like to discuss, so feel free to ask me anything!


## 2. Define Tools

In [4]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

tools = [add, multiply]

## 3. Link Tools with LLM

In [5]:
llama3_with_tools = llama3.bind_tools(tools) # tool_choice = 'any', 'actual_tool_name', 

ai_msg = llama3_with_tools.invoke("What is (121+5)*9 ?")

ai_msg.tool_calls

[{'name': 'add', 'args': {'a': 121, 'b': 5}, 'id': 'call_srwq'},
 {'name': 'multiply', 'args': {'a': 110, 'b': 9}, 'id': 'call_k7tr'}]

In [6]:
llama3_with_tools.to_json()['kwargs']['kwargs']['tools']

[{'type': 'function',
  'function': {'name': 'add',
   'description': 'add(a: int, b: int) -> int - Adds a and b.\n\n    Args:\n        a: first int\n        b: second int',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
    'required': ['a', 'b']}}},
 {'type': 'function',
  'function': {'name': 'multiply',
   'description': 'multiply(a: int, b: int) -> int - Multiplies a and b.\n\n    Args:\n        a: first int\n        b: second int',
   'parameters': {'type': 'object',
    'properties': {'a': {'type': 'integer'}, 'b': {'type': 'integer'}},
    'required': ['a', 'b']}}}]

## 4. Use LLM with Tools

In [7]:
from langchain_core.messages import HumanMessage, ToolMessage

query = "What is (121+5)*9 ?"

messages = [HumanMessage(query)]
ai_msg = llama3_with_tools.invoke(query)
messages.append(ai_msg)

for tool_call in ai_msg.tool_calls:
    selected_tool = {"add": add, "multiply": multiply}[tool_call["name"].lower()]
    tool_output = selected_tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

messages

[HumanMessage(content='What is (121+5)*9 ?'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bpvw', 'function': {'arguments': '{"a":121,"b":5}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_kskt', 'function': {'arguments': '{"a":126,"b":9}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 1104, 'total_tokens': 1228, 'prompt_time': 0.859129764, 'completion_time': 0.132251627, 'total_time': 0.991381391}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-a819a429-7c31-4bff-8721-4de91c130431-0', tool_calls=[{'name': 'add', 'args': {'a': 121, 'b': 5}, 'id': 'call_bpvw'}, {'name': 'multiply', 'args': {'a': 126, 'b': 9}, 'id': 'call_kskt'}]),
 ToolMessage(content='126', tool_call_id='call_bpvw'),
 ToolMessage(content='1134', tool_call_id='call_kskt')]

In [8]:
final_response = llama3.invoke(messages)

print(final_response.content)

Now that we have the results of both tool calls, we can calculate the final answer.

The first tool call "call_bpvw" yielded 126.

The second tool call "call_kskt" yielded 1134.

To calculate the final answer, we multiply 126 by 9, which is the input for the second tool call.

126 * 9 = 1134

So, the final answer is indeed 1134.


In [55]:
(121+5)*9

1134

## 5. Create Independent Decision Taking Agent

In [9]:
from langchain.agents import AgentExecutor, create_tool_calling_agent, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. You may not need to use tools for every query - the user may just want to chat!",
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llama3 = ChatOpenAI(api_key=groq_api_key,
                    base_url="https://api.groq.com/openai/v1/", 
                    model="llama3-70b-8192",
                    temperature=0.0
                   )

agent = create_tool_calling_agent(llama3, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [10]:
from langchain_core.messages import HumanMessage

agent_executor.invoke({"messages": [HumanMessage(content="Add 8 to 199 and then multiply by 4")]})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 199, 'b': 8}`


207
Invoking: `multiply` with `{'a': 207, 'b': 4}`


828The final answer is 828.

> Finished chain.


{'messages': [HumanMessage(content='Add 8 to 199 and then multiply by 4')],
 'output': 'The final answer is 828.'}

In [85]:
(8+199)*4

828

In [11]:
agent_executor.invoke({"messages": [HumanMessage(content="What is 3 * 12? Also, what is 11 + 49?")]})



> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'a': 3, 'b': 12}`


36
Invoking: `add` with `{'a': 11, 'b': 49}`


60The answers are 36 and 60.

> Finished chain.


{'messages': [HumanMessage(content='What is 3 * 12? Also, what is 11 + 49?')],
 'output': 'The answers are 36 and 60.'}

In [20]:
agent_executor.invoke({"messages": [HumanMessage(content="Add 121 to 5 then multiply result by 2")]})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 121, 'b': 5}`


126
Invoking: `multiply` with `{'a': 126, 'b': 2}`


252The final answer is 252.

> Finished chain.


{'messages': [HumanMessage(content='Add 121 to 5 then multiply result by 2')],
 'output': 'The final answer is 252.'}

In [13]:
(121+5) * 2

252

In [14]:
agent_executor.invoke({"messages": [HumanMessage(content="Add 121,323,445 to 453,341,321")]})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 121323445, 'b': 453341321}`


574664766The result is 574664766.

> Finished chain.


{'messages': [HumanMessage(content='Add 121,323,445 to 453,341,321')],
 'output': 'The result is 574664766.'}

In [15]:
f"{121323445+453341321:,}"

'574,664,766'

In [16]:
agent_executor.invoke({"messages": [HumanMessage(content="Add two to one hundred twenty one and then multiply result by five")]})



> Entering new AgentExecutor chain...

Invoking: `add` with `{'a': 2, 'b': 121}`


123
Invoking: `multiply` with `{'a': 123, 'b': 5}`


615The final answer is 615.

> Finished chain.


{'messages': [HumanMessage(content='Add two to one hundred twenty one and then multiply result by five')],
 'output': 'The final answer is 615.'}

In [17]:
(2+121) *5

615

## Summary

In this tutorial, I explained how to pass **functions** to **open source LLMs**. We used **Groq API** for accessing open source LLM **LLaMa-3**.